In [ ]:
# Read .litmus file into string
import numpy as np


def makeIntro(testname, i, th_writevals):
    retstr = ""
    
    retstr += "\t.section \".text\"\n"
    retstr += "\t.globl P1\n"       
    retstr += "\t.type P1, @function\n"
    retstr += "\n"
    
    retstr += "P" + i + ":\n"
    retstr += "\tpushq %rbx\n"
    retstr += "\tpushq %r12\n"
    retstr += "\tpushq %r13\n"
    retstr += "\tpushq %rbp\n"
    retstr += "\tmovq %rsp, %rbp\n"
    retstr += "\n"
    
    retstr += "\tmovq 48(%rdi), %r13\t# no iterations\n"
    retstr += "\tmovq 40(%rdi), %r11\t# buf2\n"
    retstr += "\tmovq 32(%rdi), %r10\t# buf1\n"
    retstr += "\tmovq 24(%rdi), %rcx\t# ptr to w\n"
    retstr += "\tmovq 16(%rdi), %rdx\t# ptr to z\n"
    retstr += "\tmovq 8(%rdi), %rsi\t# ptr to y\n"
    retstr += "\tmovq (%rdi), %rdi\t# ptr to x\n"
    retstr += "\tmovq $0, %r12\t# loop index\n"
    retstr += "\tmovq " + th_writevals.keys()[th_writevals.values().index("%r8, ")] + ", %r8\t# writeval 1\n"
    retstr += "\tmovq " + th_writevals.keys()[th_writevals.values().index("%r9, ")] + ", %r9\t# writeval 2\n"
    
    retstr += "\t jmp .LOOPEND\n" 
    retstr += ".LOOPSTART\n"
    retstr += "\t # " + testname + " Thread " + i + "\n"
    
    return retstr



def convertLine(thln, th_writevals, memlocs, reglocs):
    retstr = "\t"
    toks = thln.split(" ")
    
    # Is it a fence?
    if (toks[0] == "fence"):
        retstr += "MFENCE \t# fence"
        return retstr
    
    # So it's a move
    retstr += "movq "
    
    # first part (source)
    if(toks[0][0] == "$"):
        if(toks[0] in th_writevals):
            retstr += th_writevals.get(toks[0])
        elif (len(th_writevals) == 0):
            retstr += "%r8, "
            th_writevals[toks[0]] = "%r8, "
        else: 
            retstr += "%r9, "
            th_writevals[toks[0]] = "%r9, "
    elif(toks[0][0] == "["):
        retstr += memlocs.get(toks[0][1]) + ", "
    elif(toks[0][0]) == "%"):
        retstr += reglocs.get(toks[0]) + ", "
        
    # second part (destination)
    if(toks[2][0] == "["):
        retstr += memlocs.get(toks[2][1])
    elif(toks[2][0]) == "%"):
        retstr += reglocs.get(toks[2])
    
    return retstr
    
    
def makeOutro(no_writevals):
    retstr = "\n"
    
    retstr += "\t# Store in correct location in bufs\n"
    retstr += "\tmovq %rax, (%r10, %r12, 4)\n"
    retstr += "\tmovq %rbx, (%r11, %r12, 4)\n"
    retstr += "\n"
    retstr += "\t# Increment loop index and writevals\n"
    retstr += "\tincq %r12\n"
    retstr += "\t addq $" + no_writevals + ", %r8\n"
    retstr += "\t addq $" + no_writevals + ", %r9\n"
    retstr += "\n"
    retstr += ".LOOPEND:\n"
    retstr += "\tcmpq %r13,%r12\n"
    retstr += "\tjl .LOOPSTART"
    retstr += "\n"
    retstr += "\tpopq %rbp\n"
    retstr += "\tpopq %r13\n"
    retstr += "\tpopq %r12\n"
    retstr += "\tpopq %rbx\n"
    retstr += "\tret\n" 
    
    return retstr


def main():
    filename = sys.argv[1]
    testname = .split(".")
    with open(filename) as f:
        
        # Define the use of memloc/regloc registers
        memlocs = {"x":"(%rdi)", "y":"(%rsi)", "z":"(%rdx)", "w":"(%rcx)"}
        reglocs = {"%r1":"%rax", "%r2":"%rbx"}
        
        # Read in data
        no_thrs = int(f.readline())
        no_memlocs = int(f.readline())
        no_writevals = int(f.readline())
        fstr = f.read()
        
        # For each expected thread..
        for i in range(no_thrs):
            
            # Extract it
            th_start = fstr.index('~')
            th_end = fstr[th_start + 1:].index('~')
            thstr = fstr[th_start:th_end].splitlines()
            thstr = thstr[1:] # skip line with tilde
            fstr = fstr[th_end:]
            
            thstr_perp = ""            
            th_writevals = {}
            
            # For each line of this thread
            for i in range(len(thstr)):
                thln = thstr[i]
                thstr_perp += convertLine(thln, th_writevals, memlocs, reglocs)
        
        
        
            out = open(testname + "_" + i +".s", "w")
            out.write(makeIntro(testname, i, th_writevals))
            out.write(thstr_perp)
            out.write(makeOutro(no_writevals))
            out.close()
            print(testname + "_" + i +".s created successfully!\n")
            
            
        # Postcondition!!!!
        out = open(testname + "_postcondition.txt", "w")
        out.write(fstr[2:])
        out.close()
            
    print(testname + "conversion complete")    